# 📈 Week 22, Day 2: Time Series Forecasting

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gouthamgo/Learn-AI/blob/main/Phase-4-Advanced-AI/Week-22-Job-Critical-Skills/Day-2-Time-Series-Forecasting.ipynb)

## 🚀 Why This Matters

Time series forecasting is **CRITICAL** for business operations:
- 🛒 **Retail**: Demand forecasting prevents stockouts & overstocking (saves millions)
- 💰 **Finance**: Stock price prediction, risk management
- 🏭 **Manufacturing**: Production planning, inventory optimization
- ⚡ **Energy**: Load forecasting for power grids
- 🌤️ **Weather**: Temperature, rainfall prediction

**Job Market Reality:**
- Woolworths job listing: "Create time series models for demand forecasting" ✅
- Finance companies: Stock/crypto price prediction
- Tech companies: User growth, server load prediction
- Manufacturing: Equipment maintenance prediction

**Real Impact**:
- Walmart saves $billions with demand forecasting
- Amazon adjusts prices 2.5M times/day using predictions
- Airlines use forecasting for dynamic pricing

## 📋 What You'll Learn Today

1. **Time Series Fundamentals** - Trend, seasonality, stationarity
2. **Classical Methods** - ARIMA, SARIMA
3. **Modern Methods** - Prophet (Facebook's library)
4. **Deep Learning** - LSTMs, GRUs for time series
5. **Evaluation Metrics** - MAE, RMSE, MAPE
6. **🏆 Project: Retail Demand Forecasting System**

---

## Part 1: Time Series Fundamentals

### What is Time Series Data?

Data points indexed in time order:
- Stock prices (every minute)
- Daily sales (every day)
- Monthly revenue (every month)
- Server metrics (every second)

### Key Components

1. **Trend** (T): Long-term increase/decrease
2. **Seasonality** (S): Regular patterns (daily, weekly, yearly)
3. **Cyclical** (C): Long-term oscillations (business cycles)
4. **Residual/Noise** (R): Random variations

$$Y(t) = T(t) + S(t) + C(t) + R(t)$$

### Stationarity

**Stationary series**: Statistical properties don't change over time
- Constant mean
- Constant variance
- No seasonality

**Why it matters**: Most statistical methods assume stationarity!

In [ ]:
# Install required libraries
!pip install pandas numpy matplotlib seaborn statsmodels prophet pmdarima scikit-learn -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ All libraries installed!")

In [ ]:
# Create synthetic retail sales data
np.random.seed(42)

# Generate 2 years of daily sales data
dates = pd.date_range(start='2022-01-01', end='2023-12-31', freq='D')
n_days = len(dates)

# Components
trend = np.linspace(100, 150, n_days)  # Upward trend
seasonality = 20 * np.sin(2 * np.pi * np.arange(n_days) / 365)  # Yearly seasonality
weekly = 10 * np.sin(2 * np.pi * np.arange(n_days) / 7)  # Weekly seasonality
noise = np.random.normal(0, 5, n_days)  # Random noise

# Combine components
sales = trend + seasonality + weekly + noise
sales = np.maximum(sales, 0)  # No negative sales

# Create DataFrame
df = pd.DataFrame({
    'date': dates,
    'sales': sales
})

print("✅ Retail Sales Dataset Created!")
print(f"Period: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"Total days: {len(df)}")
print(f"\nSample data:")
print(df.head())

In [ ]:
# Visualize the time series
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# Original series
axes[0].plot(df['date'], df['sales'], linewidth=1)
axes[0].set_title('Original Sales Data', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Sales')
axes[0].grid(alpha=0.3)

# Trend
axes[1].plot(df['date'], trend, color='red', linewidth=2)
axes[1].set_title('Trend Component', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Trend')
axes[1].grid(alpha=0.3)

# Seasonality (yearly)
axes[2].plot(df['date'], seasonality, color='green', linewidth=2)
axes[2].set_title('Yearly Seasonality Component', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Seasonality')
axes[2].grid(alpha=0.3)

# Noise
axes[3].plot(df['date'], noise, color='gray', linewidth=0.5, alpha=0.7)
axes[3].set_title('Noise/Residual Component', fontsize=14, fontweight='bold')
axes[3].set_ylabel('Noise')
axes[3].set_xlabel('Date')
axes[3].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Time series decomposed into: Trend + Seasonality + Noise")

### Checking Stationarity

**Augmented Dickey-Fuller (ADF) Test**:
- Null hypothesis: Time series is non-stationary
- p-value < 0.05 → Reject null → Series is stationary ✅
- p-value > 0.05 → Series is non-stationary ❌

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def check_stationarity(series, name='Series'):
    """
    Perform ADF test for stationarity.
    """
    result = adfuller(series)
    
    print(f"\n{'='*50}")
    print(f"ADF Test Results for {name}")
    print(f"{'='*50}")
    print(f"ADF Statistic: {result[0]:.4f}")
    print(f"p-value: {result[1]:.4f}")
    print(f"Critical Values:")
    for key, value in result[4].items():
        print(f"  {key}: {value:.4f}")
    
    if result[1] < 0.05:
        print(f"\n✅ Series is STATIONARY (p < 0.05)")
    else:
        print(f"\n❌ Series is NON-STATIONARY (p > 0.05)")
        print("   → Need to apply differencing or transformation")

# Test original series
check_stationarity(df['sales'], 'Original Sales')

In [ ]:
# Make series stationary using differencing
df['sales_diff'] = df['sales'].diff()  # First-order differencing

# Remove NaN from differencing
df_stationary = df.dropna()

# Test differenced series
check_stationarity(df_stationary['sales_diff'], 'Differenced Sales')

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].plot(df['date'], df['sales'])
axes[0].set_title('Original Sales (Non-Stationary)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Sales')
axes[0].grid(alpha=0.3)

axes[1].plot(df_stationary['date'], df_stationary['sales_diff'])
axes[1].set_title('Differenced Sales (Stationary)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Sales Difference')
axes[1].set_xlabel('Date')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Part 2: ARIMA Models

### ARIMA: AutoRegressive Integrated Moving Average

**ARIMA(p, d, q)**:
- **p** = AR order (autoregressive): Past values influence current value
- **d** = Differencing order: Make series stationary
- **q** = MA order (moving average): Past errors influence current value

### How to Choose p, d, q?

1. **d**: Difference until stationary (usually 1 or 2)
2. **p**: Look at PACF plot (partial autocorrelation)
3. **q**: Look at ACF plot (autocorrelation)
4. **Or**: Use auto_arima to find best parameters!

### SARIMA: Seasonal ARIMA

**SARIMA(p,d,q)(P,D,Q,s)**:
- Additional seasonal parameters
- s = seasonal period (7 for weekly, 12 for monthly, 365 for yearly)
- Better for data with seasonality

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Split data into train/test
train_size = int(len(df) * 0.8)
train_df = df[:train_size].copy()
test_df = df[train_size:].copy()

print(f"Training set: {len(train_df)} days")
print(f"Test set: {len(test_df)} days")
print(f"Train period: {train_df['date'].min().date()} to {train_df['date'].max().date()}")
print(f"Test period: {test_df['date'].min().date()} to {test_df['date'].max().date()}")

In [ ]:
# Fit ARIMA model
print("Training ARIMA(2,1,2) model...")

arima_model = ARIMA(train_df['sales'], order=(2, 1, 2))
arima_fitted = arima_model.fit()

print("\n✅ ARIMA Model Trained!")
print(arima_fitted.summary())

In [ ]:
# Make predictions
n_forecast = len(test_df)
arima_forecast = arima_fitted.forecast(steps=n_forecast)

# Calculate errors
mae = mean_absolute_error(test_df['sales'], arima_forecast)
rmse = np.sqrt(mean_squared_error(test_df['sales'], arima_forecast))
mape = np.mean(np.abs((test_df['sales'].values - arima_forecast) / test_df['sales'].values)) * 100

print(f"\n📊 ARIMA Performance:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")

# Visualize
plt.figure(figsize=(14, 6))
plt.plot(train_df['date'], train_df['sales'], label='Training Data', linewidth=2)
plt.plot(test_df['date'], test_df['sales'], label='Actual Test Data', linewidth=2)
plt.plot(test_df['date'], arima_forecast, label='ARIMA Forecast', linewidth=2, linestyle='--')
plt.axvline(x=train_df['date'].iloc[-1], color='red', linestyle=':', linewidth=2, label='Train/Test Split')
plt.title('ARIMA Forecasting', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Auto ARIMA - automatically find best parameters
from pmdarima import auto_arima

print("Finding best ARIMA parameters with auto_arima...")
print("(This may take a minute)\n")

auto_model = auto_arima(
    train_df['sales'],
    start_p=0, start_q=0,
    max_p=5, max_q=5,
    d=None,  # Let auto_arima determine d
    seasonal=False,
    stepwise=True,
    suppress_warnings=True,
    error_action='ignore',
    trace=True
)

print(f"\n✅ Best model: ARIMA{auto_model.order}")
print(f"AIC: {auto_model.aic():.2f}")

### SARIMA for Seasonal Data

In [ ]:
# Fit SARIMA model with weekly seasonality
print("Training SARIMA model with weekly seasonality...")

sarima_model = SARIMAX(
    train_df['sales'],
    order=(1, 1, 1),  # ARIMA parameters
    seasonal_order=(1, 1, 1, 7)  # Seasonal parameters (weekly)
)
sarima_fitted = sarima_model.fit(disp=False)

print("✅ SARIMA Model Trained!")

# Make predictions
sarima_forecast = sarima_fitted.forecast(steps=n_forecast)

# Calculate errors
mae = mean_absolute_error(test_df['sales'], sarima_forecast)
rmse = np.sqrt(mean_squared_error(test_df['sales'], sarima_forecast))
mape = np.mean(np.abs((test_df['sales'].values - sarima_forecast) / test_df['sales'].values)) * 100

print(f"\n📊 SARIMA Performance:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")

# Visualize
plt.figure(figsize=(14, 6))
plt.plot(train_df['date'], train_df['sales'], label='Training Data', linewidth=2)
plt.plot(test_df['date'], test_df['sales'], label='Actual Test Data', linewidth=2)
plt.plot(test_df['date'], sarima_forecast, label='SARIMA Forecast', linewidth=2, linestyle='--')
plt.axvline(x=train_df['date'].iloc[-1], color='red', linestyle=':', linewidth=2, label='Train/Test Split')
plt.title('SARIMA Forecasting (Weekly Seasonality)', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Part 3: Prophet (Facebook's Time Series Library)

### Why Prophet?

**Advantages**:
- ✅ Handles seasonality automatically (daily, weekly, yearly)
- ✅ Robust to missing data
- ✅ Robust to outliers
- ✅ Includes holidays effects
- ✅ Easy to use (minimal parameter tuning)
- ✅ Interpretable components

**Used by**: Facebook, Uber, Airbnb, many tech companies

**When to use**:
- Business forecasting (sales, revenue, users)
- Data with strong seasonal patterns
- Need quick, reliable forecasts

### Prophet Model

$$y(t) = g(t) + s(t) + h(t) + \epsilon_t$$

- **g(t)**: Trend (piecewise linear or logistic)
- **s(t)**: Seasonality (Fourier series)
- **h(t)**: Holidays effects
- **ε_t**: Error term

In [ ]:
from prophet import Prophet

# Prepare data for Prophet (needs 'ds' and 'y' columns)
prophet_train = train_df.rename(columns={'date': 'ds', 'sales': 'y'})[['ds', 'y']]
prophet_test = test_df.rename(columns={'date': 'ds', 'sales': 'y'})[['ds', 'y']]

# Create and fit Prophet model
print("Training Prophet model...")

prophet_model = Prophet(
    daily_seasonality=False,
    weekly_seasonality=True,
    yearly_seasonality=True,
    seasonality_mode='additive',
    changepoint_prior_scale=0.05
)

prophet_model.fit(prophet_train)

print("✅ Prophet Model Trained!")

In [ ]:
# Make predictions
future = prophet_model.make_future_dataframe(periods=n_forecast)
prophet_forecast = prophet_model.predict(future)

# Extract test predictions
prophet_test_pred = prophet_forecast.iloc[-n_forecast:]['yhat'].values

# Calculate errors
mae = mean_absolute_error(test_df['sales'], prophet_test_pred)
rmse = np.sqrt(mean_squared_error(test_df['sales'], prophet_test_pred))
mape = np.mean(np.abs((test_df['sales'].values - prophet_test_pred) / test_df['sales'].values)) * 100

print(f"\n📊 Prophet Performance:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")

# Visualize forecast
fig = prophet_model.plot(prophet_forecast, figsize=(14, 6))
plt.axvline(x=train_df['date'].iloc[-1], color='red', linestyle=':', linewidth=2, label='Train/Test Split')
plt.title('Prophet Forecasting', fontsize=14, fontweight='bold')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Visualize components
fig = prophet_model.plot_components(prophet_forecast, figsize=(14, 10))
plt.tight_layout()
plt.show()

print("\n📊 Prophet automatically decomposed the time series into:")
print("  1. Trend: Overall direction")
print("  2. Weekly Seasonality: Day-of-week patterns")
print("  3. Yearly Seasonality: Month/season patterns")

## Part 4: Deep Learning for Time Series (LSTMs)

### Why LSTMs for Time Series?

**Long Short-Term Memory (LSTM)**:
- Can learn long-term dependencies
- Handles complex non-linear patterns
- Great for multivariate time series
- Used by: Google, Amazon, finance firms

**When to use**:
- Complex patterns ARIMA can't capture
- Multiple input features
- Large datasets (thousands of data points)
- Non-linear relationships

In [ ]:
!pip install torch -q

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

print("✅ PyTorch installed!")

In [ ]:
# Prepare data for LSTM
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_length=30):
        self.seq_length = seq_length
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.seq_length
    
    def __getitem__(self, idx):
        # Input: seq_length past values
        x = self.data[idx:idx+self.seq_length]
        # Target: next value
        y = self.data[idx+self.seq_length]
        return torch.FloatTensor(x), torch.FloatTensor([y])

# Scale data to 0-1
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_df[['sales']]).flatten()
test_scaled = scaler.transform(test_df[['sales']]).flatten()

# Create datasets
seq_length = 30  # Use 30 days to predict next day
train_dataset = TimeSeriesDataset(train_scaled, seq_length)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print(f"✅ Data prepared!")
print(f"Sequence length: {seq_length} days")
print(f"Training samples: {len(train_dataset)}")

In [ ]:
# LSTM Model
class LSTMForecaster(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, dropout=0.2):
        super(LSTMForecaster, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        # x shape: (batch, seq_length, features)
        x = x.unsqueeze(-1)  # Add feature dimension
        
        # LSTM forward pass
        lstm_out, _ = self.lstm(x)
        
        # Take last output
        last_output = lstm_out[:, -1, :]
        
        # Fully connected layer
        prediction = self.fc(last_output)
        
        return prediction

# Initialize model
model = LSTMForecaster(input_size=1, hidden_size=64, num_layers=2, dropout=0.2)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("✅ LSTM Model Created!")
print(model)

In [ ]:
# Train LSTM
num_epochs = 50
losses = []

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    for x_batch, y_batch in train_loader:
        # Forward pass
        predictions = model(x_batch)
        loss = criterion(predictions, y_batch)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    losses.append(avg_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.6f}")

print("\n✅ LSTM Training Complete!")

# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.title('LSTM Training Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Make predictions with LSTM
model.eval()

# Use last seq_length points from training to start predictions
lstm_predictions = []
current_seq = train_scaled[-seq_length:].tolist()

with torch.no_grad():
    for _ in range(len(test_df)):
        # Predict next value
        x = torch.FloatTensor(current_seq).unsqueeze(0)
        pred = model(x).item()
        lstm_predictions.append(pred)
        
        # Update sequence (sliding window)
        current_seq = current_seq[1:] + [pred]

# Inverse transform to original scale
lstm_predictions = scaler.inverse_transform(np.array(lstm_predictions).reshape(-1, 1)).flatten()

# Calculate errors
mae = mean_absolute_error(test_df['sales'], lstm_predictions)
rmse = np.sqrt(mean_squared_error(test_df['sales'], lstm_predictions))
mape = np.mean(np.abs((test_df['sales'].values - lstm_predictions) / test_df['sales'].values)) * 100

print(f"\n📊 LSTM Performance:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")

# Visualize
plt.figure(figsize=(14, 6))
plt.plot(train_df['date'], train_df['sales'], label='Training Data', linewidth=2)
plt.plot(test_df['date'], test_df['sales'], label='Actual Test Data', linewidth=2)
plt.plot(test_df['date'], lstm_predictions, label='LSTM Forecast', linewidth=2, linestyle='--')
plt.axvline(x=train_df['date'].iloc[-1], color='red', linestyle=':', linewidth=2, label='Train/Test Split')
plt.title('LSTM Forecasting', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Part 5: Model Comparison

In [ ]:
# Compare all models
# (Re-run predictions if needed to get all forecasts)

results = pd.DataFrame({
    'Model': ['ARIMA', 'SARIMA', 'Prophet', 'LSTM'],
    'MAE': [7.89, 6.45, 5.23, 4.87],  # Example values - replace with your actual values
    'RMSE': [9.45, 8.12, 6.78, 6.34],
    'MAPE': [6.2, 5.1, 4.3, 3.9]
})

print("\n📊 Model Comparison:\n")
print(results.to_string(index=False))

# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

metrics = ['MAE', 'RMSE', 'MAPE']
colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']

for i, metric in enumerate(metrics):
    axes[i].bar(results['Model'], results[metric], color=colors)
    axes[i].set_title(f'{metric} Comparison', fontsize=12, fontweight='bold')
    axes[i].set_ylabel(metric)
    axes[i].grid(axis='y', alpha=0.3)
    
    # Add value labels
    for j, v in enumerate(results[metric]):
        axes[i].text(j, v + 0.2, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🏆 Best Model (Lowest Error): LSTM!")

## 🏆 CAPSTONE PROJECT: Retail Demand Forecasting System

### Project Goal
Build a production-ready demand forecasting system for a retail chain.

### Requirements
1. Multiple products with different sales patterns
2. Handle seasonality (weekly, yearly)
3. Incorporate external factors (promotions, holidays)
4. Compare multiple models
5. Generate actionable insights
6. Visualize forecasts with confidence intervals

In [ ]:
# Create multi-product retail dataset
np.random.seed(42)

products = ['Electronics', 'Clothing', 'Food', 'Home_Goods']
dates = pd.date_range(start='2021-01-01', end='2023-12-31', freq='D')

retail_data = []

for product in products:
    n_days = len(dates)
    
    # Different patterns for each product
    if product == 'Electronics':
        base = 200
        trend = np.linspace(0, 50, n_days)
        # Strong yearly seasonality (Christmas spike)
        seasonality = 30 * np.sin(2 * np.pi * (np.arange(n_days) % 365) / 365 - np.pi/2) + 30
    elif product == 'Clothing':
        base = 150
        trend = np.linspace(0, 30, n_days)
        # Seasonal (summer/winter)
        seasonality = 20 * np.sin(2 * np.pi * np.arange(n_days) / 365)
    elif product == 'Food':
        base = 500
        trend = np.linspace(0, 20, n_days)
        # Weekly seasonality (weekend shopping)
        seasonality = 50 * np.sin(2 * np.pi * np.arange(n_days) / 7)
    else:  # Home_Goods
        base = 100
        trend = np.linspace(0, 40, n_days)
        seasonality = 15 * np.sin(2 * np.pi * np.arange(n_days) / 365)
    
    # Add promotions (random spikes)
    promotions = np.zeros(n_days)
    promo_days = np.random.choice(n_days, size=20, replace=False)
    promotions[promo_days] = np.random.uniform(50, 100, 20)
    
    # Combine
    noise = np.random.normal(0, 10, n_days)
    sales = base + trend + seasonality + promotions + noise
    sales = np.maximum(sales, 0)
    
    # Create records
    for i, date in enumerate(dates):
        retail_data.append({
            'date': date,
            'product': product,
            'sales': sales[i],
            'promotion': 1 if promotions[i] > 0 else 0
        })

retail_df = pd.DataFrame(retail_data)

print("✅ Multi-Product Retail Dataset Created!")
print(f"Products: {products}")
print(f"Period: {retail_df['date'].min().date()} to {retail_df['date'].max().date()}")
print(f"Total records: {len(retail_df)}")
print(f"\nSample data:")
print(retail_df.head(12))

In [ ]:
# Visualize sales by product
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for i, product in enumerate(products):
    product_data = retail_df[retail_df['product'] == product]
    axes[i].plot(product_data['date'], product_data['sales'], linewidth=1)
    
    # Highlight promotions
    promo_data = product_data[product_data['promotion'] == 1]
    axes[i].scatter(promo_data['date'], promo_data['sales'], 
                    color='red', s=30, alpha=0.6, label='Promotion')
    
    axes[i].set_title(f'{product} Sales', fontsize=12, fontweight='bold')
    axes[i].set_ylabel('Sales')
    axes[i].set_xlabel('Date')
    axes[i].legend()
    axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Forecast for each product using Prophet
def forecast_product(product_name, retail_df, forecast_days=90):
    """
    Forecast sales for a specific product.
    """
    # Filter product data
    product_data = retail_df[retail_df['product'] == product_name].copy()
    
    # Train/test split
    train_size = len(product_data) - forecast_days
    train = product_data[:train_size]
    test = product_data[train_size:]
    
    # Prepare for Prophet
    prophet_data = train.rename(columns={'date': 'ds', 'sales': 'y'})[['ds', 'y', 'promotion']]
    
    # Create model with promotion as regressor
    model = Prophet(
        weekly_seasonality=True,
        yearly_seasonality=True,
        seasonality_mode='additive'
    )
    model.add_regressor('promotion')
    
    # Fit model
    model.fit(prophet_data)
    
    # Create future dataframe
    future = model.make_future_dataframe(periods=forecast_days)
    future['promotion'] = 0  # No promotions in future (can be customized)
    
    # Merge actual promotions from test set
    test_promo = test.rename(columns={'date': 'ds'})[['ds', 'promotion']]
    future = future.merge(test_promo, on='ds', how='left', suffixes=('', '_actual'))
    future['promotion'] = future['promotion_actual'].fillna(future['promotion'])
    future = future.drop('promotion_actual', axis=1)
    
    # Forecast
    forecast = model.predict(future)
    
    # Extract test predictions
    test_pred = forecast.iloc[-forecast_days:]['yhat'].values
    
    # Calculate metrics
    mae = mean_absolute_error(test['sales'], test_pred)
    rmse = np.sqrt(mean_squared_error(test['sales'], test_pred))
    mape = np.mean(np.abs((test['sales'].values - test_pred) / test['sales'].values)) * 100
    
    return {
        'model': model,
        'forecast': forecast,
        'train': train,
        'test': test,
        'test_pred': test_pred,
        'mae': mae,
        'rmse': rmse,
        'mape': mape
    }

# Forecast all products
print("Forecasting sales for all products...\n")
forecasts = {}

for product in products:
    print(f"Processing {product}...")
    forecasts[product] = forecast_product(product, retail_df, forecast_days=90)
    print(f"  MAE: {forecasts[product]['mae']:.2f}")
    print(f"  RMSE: {forecasts[product]['rmse']:.2f}")
    print(f"  MAPE: {forecasts[product]['mape']:.2f}%\n")

print("✅ All forecasts completed!")

In [ ]:
# Visualize forecasts for all products
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, product in enumerate(products):
    result = forecasts[product]
    
    # Plot training data
    axes[i].plot(result['train']['date'], result['train']['sales'], 
                 label='Training Data', linewidth=2)
    
    # Plot actual test data
    axes[i].plot(result['test']['date'], result['test']['sales'], 
                 label='Actual', linewidth=2, color='green')
    
    # Plot forecast
    test_dates = result['test']['date'].values
    axes[i].plot(test_dates, result['test_pred'], 
                 label='Forecast', linewidth=2, linestyle='--', color='red')
    
    # Plot confidence interval
    forecast_test = result['forecast'].iloc[-90:]
    axes[i].fill_between(
        test_dates,
        forecast_test['yhat_lower'].values,
        forecast_test['yhat_upper'].values,
        alpha=0.2, color='red'
    )
    
    axes[i].axvline(x=result['train']['date'].iloc[-1], 
                    color='black', linestyle=':', linewidth=2)
    
    axes[i].set_title(f"{product} - MAPE: {result['mape']:.2f}%", 
                      fontsize=12, fontweight='bold')
    axes[i].set_ylabel('Sales')
    axes[i].set_xlabel('Date')
    axes[i].legend()
    axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Generate actionable insights
print("\n" + "="*70)
print("🎯 RETAIL DEMAND FORECASTING - ACTIONABLE INSIGHTS")
print("="*70 + "\n")

for product in products:
    result = forecasts[product]
    
    # Calculate average forecast
    avg_forecast = np.mean(result['test_pred'])
    avg_actual = np.mean(result['test']['sales'])
    
    # Calculate total forecast
    total_forecast = np.sum(result['test_pred'])
    
    # Forecast accuracy
    accuracy = 100 - result['mape']
    
    print(f"📦 {product.upper()}")
    print("-" * 70)
    print(f"  Forecast Accuracy: {accuracy:.1f}%")
    print(f"  Average Daily Sales (Next 90 days): {avg_forecast:.0f} units")
    print(f"  Total Forecast (Next 90 days): {total_forecast:.0f} units")
    
    # Recommendations
    if avg_forecast > avg_actual * 1.1:
        print(f"  📈 RECOMMENDATION: Increase inventory by 10-15%")
    elif avg_forecast < avg_actual * 0.9:
        print(f"  📉 RECOMMENDATION: Reduce inventory by 10-15%")
    else:
        print(f"  ✅ RECOMMENDATION: Maintain current inventory levels")
    
    # Peak day
    peak_day = result['test']['date'].iloc[np.argmax(result['test_pred'])]
    peak_value = np.max(result['test_pred'])
    print(f"  🔥 Peak Demand Day: {peak_day.date()} ({peak_value:.0f} units)")
    print()

print("="*70)

## 🎓 Key Takeaways

### What You've Learned

1. **Time Series Fundamentals**
   - Trend, seasonality, stationarity
   - Decomposition of time series
   - ADF test for stationarity

2. **Classical Methods**
   - ARIMA for non-seasonal data
   - SARIMA for seasonal patterns
   - auto_arima for parameter selection

3. **Modern Methods**
   - Prophet for business forecasting
   - Automatic seasonality detection
   - Holiday effects and external regressors

4. **Deep Learning**
   - LSTMs for complex patterns
   - Sequence modeling
   - When to use deep learning vs classical

5. **Evaluation**
   - MAE, RMSE, MAPE metrics
   - Model comparison
   - Business impact assessment

6. **Production System**
   - Multi-product forecasting
   - Confidence intervals
   - Actionable recommendations

### Interview-Ready Skills ✅

**You can now answer:**
- "Explain the difference between ARIMA and Prophet"
- "How do you check if a time series is stationary?"
- "When would you use LSTMs over ARIMA?"
- "How do you handle seasonality in forecasting?"
- "Explain a demand forecasting system you've built"

**You can now build:**
- Retail demand forecasting (Woolworths-style)
- Financial time series prediction
- User growth forecasting
- Server load prediction

### Real-World Applications

**Retail** (Woolworths, Amazon):
- Demand forecasting for inventory
- Price optimization
- Staffing requirements

**Finance**:
- Stock price prediction
- Risk forecasting
- Cash flow prediction

**Tech**:
- User growth forecasting
- Resource planning (servers, bandwidth)
- Anomaly detection

**Manufacturing**:
- Production planning
- Equipment maintenance
- Supply chain optimization

---

## 🚀 Next Steps

1. **Add to portfolio**: Deploy demand forecasting as web dashboard
2. **Learn more**: VAR (multivariate), GARCH (volatility), Transformer models
3. **Practice**: Kaggle time series competitions
4. **Scale**: Learn Spark MLlib for distributed forecasting

---

## 📚 Additional Resources

**Libraries**:
- Prophet: Facebook's forecasting library
- pmdarima: Auto ARIMA
- statsmodels: Classical time series models
- darts: Modern time series library
- GluonTS: Deep learning time series (Amazon)

**Books**:
- "Forecasting: Principles and Practice" by Hyndman & Athanasopoulos
- "Time Series Analysis" by Hamilton

**Papers**:
- "Forecasting at Scale" (Prophet paper - Facebook)
- "N-BEATS" (Neural basis expansion - state-of-the-art)

---

## ✅ Job Market Relevance

**Woolworths ML Engineer**: ✅
> "Create time series models for demand forecasting"

**Industry Impact**: ✅
- Walmart saves $billions with forecasting
- Amazon adjusts 2.5M prices daily
- Every retail/finance company needs this

**You are now job-ready for time series roles!** 🎉

---

**Congratulations!** You've completed Day 2 of Week 22. Tomorrow: Apache Airflow & ML Orchestration! 🚀